# TP 1: LDA/QDA y optimización matemática de modelos

# Intro teórica

## Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

## Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

## LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

## Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

## Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

# Código provisto

Con el fin de no retrasar al alumno con cuestiones estructurales y/o secundarias al tema que se pretende tratar, se provee una base de código que **no es obligatoria de usar** pero se asume que resulta resulta beneficiosa.

In [1]:
import numpy as np
import pandas as pd
import numpy.linalg as LA
from scipy.linalg import cholesky, solve_triangular
from scipy.linalg.lapack import dtrtri

## Base code

In [2]:
class BaseBayesianClassifier:
  def __init__(self):
    pass

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    
    # Q3: para que sirve bincount?
    """ 
    bincount cuenta la cantidad de ocurrencias de cada valor entero no negativo en el array.
    En este caso, cuenta cuántas muestras hay de cada clase (0, 1, 2, ... k-1).
    Al dividir por y.size (total de muestras), obtenemos la frecuencia relativa de cada clase,
    que es el estimador de la probabilidad a priori pi_j. 
    """
    return np.log(a_priori)

  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?
    """ 
    _fit_params va al final porque algunas implementaciones (como QDA) necesitan saber la cantidad 
    de clases k para inicializar o estructurar sus parámetros (ej. listas de covarianzas).
    La cantidad de clases se infiere o guarda implícitamente al calcular el a priori (self.log_a_priori).
    Si se moviera antes, self.log_a_priori podría no existir, fallando el entrenamiento que dependa de 
    ello.
    """ 

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=int)

    for i in range(m_obs):
      y_hat[i] = self._predict_one(X[:,i].reshape(-1,1))

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [3]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    self.inv_covs = [LA.inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]
    
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    """ 
    'y' viene con shape (n, 1) (vector columna). La comparación y==idx genera una máscara (n, 1).
    Sin embargo, para indexar columnas en X (que es (p, n)), numpy espera un array 1D de booleanos para 
    la dimensión de columnas.
    y.flatten() convierte (n, 1) a (n,), permitiendo el slicing correcto X[:, mask_1d].
    """ 
    
    # Q6: por que se usa bias=True en vez del default bias=False?

    """ 
    bias=True normaliza por N (poblacional o Máxima Verosimilitud), mientras que bias=False normaliza 
    por N-1 (metodo insesgado).
    En esquemas de Máxima Verosimilitud (como los derivados teoricamente para QDA/LDA gaussianos), 
    el estimador MLE de la covarianza divide por N.
    """ 
    
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    
    # Q7: que hace axis=1? por que no axis=0?
    """ 
    Axis=1 calcula el promedio a lo largo de las columnas (las observaciones).
    X tiene shape (features, observaciones). Queremos el vector promedio de features (p, 1).
    Si usaramos axis=0 promediaríamos los features entre sí, lo cual no tendría sentido físico.
    """ 
      
  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(LA.det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [4]:
class TensorizedQDA(QDA):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X,y)

        # stack onto new dimension
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x

        return 0.5*np.log(LA.det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()

    def _predict_one(self, x):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

In [5]:
class QDA_Chol1(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        LA.inv(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True))
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

In [6]:
class QDA_Chol2(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.Ls = [
        cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True)
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L = self.Ls[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = solve_triangular(L, unbiased_x, lower=True)

    return -np.log(L.diagonal().prod()) -0.5 * (y**2).sum()

In [7]:
class QDA_Chol3(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        dtrtri(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True), lower=1)[0]
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

## Datasets

Observar que se proveen **4 datasets diferentes**, el código de ejemplo usa uno solo pero eso no significa que ustedes se limiten al mismo. También pueden usar otros datasets de su elección.

In [8]:
from sklearn.datasets import load_iris, fetch_openml, load_wine
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

def get_penguins_dataset():
    # get data
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True, parser='auto')

    # drop non-numeric columns
    df.drop(columns=["island","sex"], inplace=True)

    # drop rows with missing values
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]

    return df.values, tgt.to_numpy().reshape(-1,1)

def get_wine_dataset():
    # get data
    data = load_wine()
    X_full = data.data
    y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
    return X_full, y_full

def get_letters_dataset():
    # get data
    letter = fetch_openml('letter', version=1, as_frame=False)
    return letter.data, letter.target.reshape(-1,1)

def label_encode(y_full):
    return LabelEncoder().fit_transform(y_full.flatten()).reshape(y_full.shape)

def split_transpose(X, y, test_size, random_state):
    # X_train, X_test, y_train, y_test but all transposed
    return [elem.T for elem in train_test_split(X, y, test_size=test_size, random_state=random_state)]

## Benchmarking

Nota: esta clase fue creada bastante rápido y no pretende ser una plataforma súper confiable sobre la que basarse, sino más bien una herramienta simple con la que poder medir varios runs y agregar la información.

En forma rápida, `warmup` es la cantidad de runs para warmup, `mem_runs` es la cantidad de runs en las que se mide el pico de uso de RAM y `n_runs` es la cantidad de runs en las que se miden tiempos.

La razón por la que se separan es que medir memoria hace ~2.5x más lento cada run, pero al mismo tiempo se estabiliza mucho más rápido.

**Importante:** tener en cuenta que los modelos que predicen en batch (usan `predict` directamente) deberían consumir, como mínimo, $n$ veces la memoria de los que predicen por observación.

In [9]:
import time
from tqdm.notebook import tqdm
from numpy.random import RandomState
import tracemalloc

RNG_SEED = 6553

class Benchmark:
    def __init__(self, X, y, n_runs=1000, warmup=100, mem_runs=100, test_sz=0.3, rng_seed=RNG_SEED, same_splits=True):
        self.X = X
        self.y = y
        self.n = n_runs
        self.warmup = warmup
        self.mem_runs = mem_runs
        self.test_sz = test_sz
        self.det = same_splits
        if self.det:
            self.rng_seed = rng_seed
        else:
            self.rng = RandomState(rng_seed)

        self.data = dict()

        print("Benching params:")
        print("Total runs:",self.warmup+self.mem_runs+self.n)
        print("Warmup runs:",self.warmup)
        print("Peak Memory usage runs:", self.mem_runs)
        print("Running time runs:", self.n)
        approx_test_sz = int(self.y.size * self.test_sz)
        print("Train size rows (approx):",self.y.size - approx_test_sz)
        print("Test size rows (approx):",approx_test_sz)
        print("Test size fraction:",self.test_sz)

    def bench(self, model_class, **kwargs):
        name = model_class.__name__
        time_data = np.empty((self.n, 3), dtype=float)  # train_time, test_time, accuracy
        mem_data = np.empty((self.mem_runs, 2), dtype=float)  # train_peak_mem, test_peak_mem
        rng = RandomState(self.rng_seed) if self.det else self.rng


        for i in range(self.warmup):
            # Instantiate model with error check for unsupported parameters
            model = model_class(**kwargs)

            # Generate current train-test split
            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )
            # Run training and prediction (timing or memory measurement not recorded)
            model.fit(X_train, y_train)
            model.predict(X_test)

        for i in tqdm(range(self.mem_runs), total=self.mem_runs, desc=f"{name} (MEM)"):

            model = model_class(**kwargs)

            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )

            tracemalloc.start()

            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()

            _, train_peak = tracemalloc.get_traced_memory()
            tracemalloc.reset_peak()

            model.predict(X_test)
            t3 = time.perf_counter()
            _, test_peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()

            mem_data[i,] = (
                train_peak / (1024 * 1024),
                test_peak / (1024 * 1024)
            )

        for i in tqdm(range(self.n), total=self.n, desc=f"{name} (TIME)"):
            model = model_class(**kwargs)

            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )

            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()
            preds = model.predict(X_test)
            t3 = time.perf_counter()

            time_data[i,] = (
                (t2 - t1) * 1000,
                (t3 - t2) * 1000,
                (y_test.flatten() == preds.flatten()).mean()
            )

        self.data[name] = (time_data, mem_data)

    def summary(self, baseline=None):
        aux = []
        for name, (time_data, mem_data) in self.data.items():
            result = {
                'model': name,
                'train_median_ms': np.median(time_data[:, 0]),
                'train_std_ms': time_data[:, 0].std(),
                'test_median_ms': np.median(time_data[:, 1]),
                'test_std_ms': time_data[:, 1].std(),
                'mean_accuracy': time_data[:, 2].mean(),
                'train_mem_median_mb': np.median(mem_data[:, 0]),
                'train_mem_std_mb': mem_data[:, 0].std(),
                'test_mem_median_mb': np.median(mem_data[:, 1]),
                'test_mem_std_mb': mem_data[:, 1].std()
            }
            aux.append(result)
        df = pd.DataFrame(aux).set_index('model')

        if baseline is not None and baseline in self.data:
            df['train_speedup'] = df.loc[baseline, 'train_median_ms'] / df['train_median_ms']
            df['test_speedup'] = df.loc[baseline, 'test_median_ms'] / df['test_median_ms']
            df['train_mem_reduction'] = df.loc[baseline, 'train_mem_median_mb'] / df['train_mem_median_mb']
            df['test_mem_reduction'] = df.loc[baseline, 'test_mem_median_mb'] / df['test_mem_median_mb']
        return df

## Ejemplo

In [10]:
# levantamos el dataset Wine, que tiene 13 features y 178 observaciones en total
X_full, y_full = get_wine_dataset()

X_full.shape, y_full.shape

((178, 13), (178, 1))

In [11]:
# encodeamos a número las clases
y_full_encoded = label_encode(y_full)

y_full[:5], y_full_encoded[:5]

(array([['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0']], dtype='<U7'),
 array([[0],
        [0],
        [0],
        [0],
        [0]]))

In [12]:
# generamos el benchmark
# observar que son valores muy bajos de runs para que corra rápido ahora
b = Benchmark(
    X_full, y_full_encoded,
    n_runs = 100,
    warmup = 20,
    mem_runs = 20,
    test_sz = 0.3,
    same_splits = False
)

Benching params:
Total runs: 140
Warmup runs: 20
Peak Memory usage runs: 20
Running time runs: 100
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3


In [13]:
# bencheamos un par
to_bench = [QDA]

for model in to_bench:
    b.bench(model)

QDA (MEM):   0%|          | 0/20 [00:00<?, ?it/s]

QDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
# como es una clase, podemos seguir bencheando más después
b.bench(TensorizedQDA)

TensorizedQDA (MEM):   0%|          | 0/20 [00:00<?, ?it/s]

TensorizedQDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
# hacemos un summary
b.summary()

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb
model,,,,,,,,,
QDA,0.085878,0.039403,0.912513,0.123337,0.982407,0.018471,0.000607,0.007697,0.000476
TensorizedQDA,0.077036,0.017017,0.407823,0.015729,0.982593,0.018471,0.000733,0.012123,0.000068


In [16]:
# son muchos datos! nos quedamos con un par nomás
summ = b.summary()

# como es un pandas DataFrame, subseteamos columnas fácil
summ[['train_median_ms', 'test_median_ms','mean_accuracy']]

,train_median_ms,test_median_ms,mean_accuracy
model,,,
QDA,0.085878,0.912513,0.982407
TensorizedQDA,0.077036,0.407823,0.982593


In [17]:
# podemos setear un baseline para que fabrique columnas de comparación
summ = b.summary(baseline='QDA')

summ

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,,,,,,,
QDA,0.085878,0.039403,0.912513,0.123337,0.982407,0.018471,0.000607,0.007697,0.000476,1.000000,1.000000,1.0,1.000000
TensorizedQDA,0.077036,0.017017,0.407823,0.015729,0.982593,0.018471,0.000733,0.012123,0.000068,1.114764,2.237522,1.0,0.634912


In [18]:
summ[[
    'train_median_ms', 'test_median_ms','mean_accuracy',
    'train_speedup', 'test_speedup',
    'train_mem_reduction', 'test_mem_reduction'
]]

,train_median_ms,test_median_ms,mean_accuracy,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,
QDA,0.085878,0.912513,0.982407,1.000000,1.000000,1.0,1.000000
TensorizedQDA,0.077036,0.407823,0.982593,1.114764,2.237522,1.0,0.634912


# Consigna QDA

**Notación**: en general notamos

* $k$ la cantidad de clases
* $n$ la cantidad de observaciones
* $p$ la cantidad de features/variables/predictores

**Sugerencia:** combinaciones adecuadas de `transpose`, `stack`, `reshape` y, ocasionalmente, `flatten` y `diagonal` suele ser más que suficiente. Se recomienda *fuertemente* explorar la dimensionalidad de cada elemento antes de implementar las clases.

## Tensorización

En esta sección nos vamos a ocupar de hacer que el modelo sea más rápido para generar predicciones, observando que incurre en un doble `for` dado que predice en forma individual un escalar para cada observación, para cada clase. Paralelizar ambos vía tensorización suena como una gran vía de mejora de tiempos.

### 1) Diferencias entre `QDA`y `TensorizedQDA`

1. ¿Sobre qué paraleliza `TensorizedQDA`? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?
2. Analizar los shapes de ``tensor_inv_cov`s` y `tensor_means` y explicar paso a paso cómo es que `TensorizedQDA` llega a predecir lo mismo que `QDA`.

### 2) Optimización

Debido a la forma cuadrática de QDA, no se puede predecir para $n$ observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de $n \times n$ en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

3. Implementar el modelo `FasterQDA` (se recomienda heredarlo de `TensorizedQDA`) de manera de eliminar el ciclo for en el método predict.
4. Mostrar dónde aparece la mencionada matriz de $n \times n$, donde $n$ es la cantidad de observaciones a predecir.
5. Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de $n \times p$. También se puede usar, de forma equivalente,
$$
np.sum(A^T \odot B, axis=0).T
$$
queda a preferencia del alumno cuál usar.

6. Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente en un nuevo modelo `EfficientQDA`.

7. Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

## Cholesky

Hasta ahora todos los esfuerzos fueron enfocados en realizar una predicción más rápida. Los tiempos de entrenamiento (teóricos al menos) siguen siendo los mismos o hasta (minúsculamente) peores, dado que todas las mejoras siguen llamando al método `_fit_params` original de `QDA`.

La descomposición/factorización de [Cholesky](https://en.wikipedia.org/wiki/Cholesky_decomposition#Statement) permite factorizar una matriz definida positiva $A = LL^T$ donde $L$ es una matriz triangular inferior. En particular, si bien se asume que $p \ll n$, invertir la matriz de covarianzas $\Sigma$ para cada clase impone un cuello de botella que podría alivianarse. Teniendo en cuenta que las matrices de covarianza son simétricas y salvo degeneración, definidas positivas, Cholesky como mínimo debería permitir invertir la matriz más rápido.

*Nota: observar que calcular* $A^{-1}b$ *equivale a resolver el sistema* $Ax=b$.

### 3) Diferencias entre implementaciones de `QDA_Chol`

8. Si una matriz $A$ tiene fact. de Cholesky $A=LL^T$, expresar $A^{-1}$ en términos de $L$. ¿Cómo podría esto ser útil en la forma cuadrática de QDA?
7. Explicar las diferencias entre `QDA_Chol1`y `QDA` y cómo `QDA_Chol1` llega, paso a paso, hasta las predicciones.
8. ¿Cuáles son las diferencias entre `QDA_Chol1`, `QDA_Chol2` y `QDA_Chol3`?
9. Comparar la performance de las 7 variantes de QDA implementadas hasta ahora ¿Qué se observa?¿Hay alguna de las implementaciones de `QDA_Chol` que sea claramente mejor que las demás?¿Alguna que sea peor?

### 4) Optimización

12. Implementar el modelo `TensorizedChol` paralelizando sobre clases/observaciones según corresponda. Se recomienda heredarlo de alguna de las implementaciones de `QDA_Chol`, aunque la elección de cuál de ellas queda a cargo del alumno según lo observado en los benchmarks de puntos anteriores.
13. Implementar el modelo `EfficientChol` combinando los insights de `EfficientQDA` y `TensorizedChol`. Si se desea, se puede implementar `FasterChol` como ayuda, pero no se contempla para el punto.
13. Comparar la performance de las 9 variantes de QDA implementadas ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

## Importante:

Las métricas que se observan al realizar benchmarking son muy dependientes del código que se ejecuta, y por tanto de las versiones de las librerías utilizadas. Una forma de unificar esto es utilizando un gestor de versiones y paquetes como _uv_ o _Poetry_, otra es simplemente usando una misma VM como la que provee Colab.

**Cada equipo debe informar las versiones de Python, NumPy y SciPy con que fueron obtenidos los resultados. En caso de que sean múltiples, agregar todos los casos**. La siguiente celda provee una ayuda para hacerlo desde un notebook, aunque como es una secuencia de comandos también sirve para consola.

In [19]:
%%bash
python --version
pip freeze | grep -E "scipy|numpy"

Python 3.12.3
numpy==2.3.1
scipy==1.16.0


**Comentario:** yo utilicé los siguientes parámetros para mi run de prueba. Esto NO significa que ustedes tengan que usar los mismos, tampoco el mismo dataset. Se agregó al notebook simplemente porque fue una pregunta común en cohortes anteriores.

In [20]:
# dataset de letters
X_letter, y_letter = get_letters_dataset()

# encoding de labels
y_letter_encoded = label_encode(y_letter.reshape(-1,1))

# instanciacion del benchmark
b = Benchmark(
    X_letter, y_letter_encoded,
    same_splits=False,
    n_runs=100,
    warmup=20,
    mem_runs=30,
    test_sz=0.2
)

Benching params:
Total runs: 150
Warmup runs: 20
Peak Memory usage runs: 30
Running time runs: 100
Train size rows (approx): 16000
Test size rows (approx): 4000
Test size fraction: 0.2


---

# Resolución del TP

## Respuestas Consigna QDA

### Diferencias entre `QDA`y `TensorizedQDA`

**1. ¿Sobre qué paraleliza `TensorizedQDA`? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?**

TensorizedQDA paraleliza sobre las k CLASES. `tensor_inv_cov` tiene shape (k, p, p) y `tensor_means` (k, p, 1).
Al operar, calcula simultáneamente el término cuadrático para todas las clases usando broadcasting de numpy.
Sin embargo, el método 'predict' sigue iterando observación por observación (ciclo for en BaseBayesianClassifier).

<br>

**2. Analizar los shapes de `tensor_inv_covs` y `tensor_means` y explicar paso a paso cómo es que `TensorizedQDA` llega a predecir lo mismo que `QDA`.**

- `x` tiene shape \((p, 1)\).
- `tensor_means` tiene shape \( (k, p, 1) \).
- `unbiased_x` = $x$ - `tensor_means`: Broadcasting resta $x$ a cada media de clase -> Resultado shape  \( (k, p, 1) \).
- `tensor_inv_cov` tiene shape \( (k, p, p) \).
- inner_prod:
    `unbiased_x`*`transpose`\((0,2,1)\) es \( (k, 1, p) \).
    \( (k, 1, p) @ (k, p, p) -> (k, 1, p) \).
    \( (k, 1, p) @ (k, p, 1) -> (k, 1, 1) \) (escalar por clase)
- Resultado final: Un vector de $(k,)$ con los valores `log-condicionales` para esa observación $x$.

<br>

**3. Implementar el modelo `FasterQDA` (se recomienda heredarlo de `TensorizedQDA`) de manera de eliminar el ciclo for en el método predict.**

In [ ]:
class FasterQDA(TensorizedQDA):
    def predict(self, X):
        # Optimización: eliminar el ciclo for sobre observaciones.
        # X tiene shape (p, n)
        n_obs = X.shape[1]

        # Precomputar log-dets: shape (k,)
        log_dets = 0.5 * np.log(LA.det(self.tensor_inv_cov)) # shape (k,)

        # Broadcasting para centrar datos:
        # self.tensor_means es (k, p, 1)
        # X es (p, n). Necesitamos expandir X para que reste con means.
        # X_expanded: (1, p, n)
        # unbiased_x: (k, p, n)
        unbiased_x = X - self.tensor_means

        # Forma cuadrática vectorizada: (x-mu)^T Sinv (x-mu)
        # unbiased_x^T : (k, n, p)
        # inv_cov      : (k, p, p)
        # term         : (k, n, p) @ (k, p, p) -> (k, n, p)
        # result       : (k, n, p) @ (k, p, n) -> (k, n, n) <-- MATRIZ GRANDE (n, n)
        # Queremos solo la diagonal de esa matriz (n, n) para cada clase k.
        
        # Implementación "Naive" (FasterQDA pero con matriz intermedia grande)
        
        # Precaución: regla de broadcast. unbiased_x es (k, p, n). Transponemos a (k, n, p) para la multiplicación.
        term1 = unbiased_x.transpose(0, 2, 1) @ self.tensor_inv_cov # (k, n, p)
        quadratic_matrix = term1 @ unbiased_x # (k, n, n)
        
        # Extraemos diagonal: (k, n)
        quad_term = np.diagonal(quadratic_matrix, axis1=1, axis2=2) # (k, n)

        # Log prob: log_det (k, 1) - 0.5 * quad_term (k, n)
        log_conditionals = log_dets.reshape(-1, 1) - 0.5 * quad_term # (k, n)

        # Sumar a priori: (k, 1) + (k, n) -> (k, n)
        log_posteriori = self.log_a_priori.reshape(-1, 1) + log_conditionals

        # Argmax sobre clases (axis 0)
        y_hat = np.argmax(log_posteriori, axis=0) # (n,)
        
        return y_hat.reshape(1, -1)

**4. Mostrar dónde aparece la mencionada matriz de  $n \times n$., donde $n$ es la cantidad de observaciones a predecir.**

Aparece en la multiplicación `term1 @ unbiased_x` dentro de FasterQDA.
Al hacer \((k, n, p) @ (k, p, n)\), el resultado es \((k, n, n)\).

Esta matriz contiene el producto interno cruzado entre la observación $i$ y la observación $j$.

Solo nos interesan los términos $i=j$ (la diagonal), que representan la distancia de Mahalanobis de $x_i$ consigo mismo.

Los términos fuera de la diagonal ($i \neq j$) son inútiles para la clasificación y gastan memoria \(O(n^2)\).

<br>

**5. Demostrar que:**

$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de $n \times p$. También se puede usar, de forma equivalente,
$$
np.sum(A^T \odot B, axis=0).T
$$

<br>

**Demostración:**

Sea $C = A \cdot B$. El elemento diagonal $C_{ii}$ es la fila $i$ de $A$ por la columna $i$ de $B$.

$$
C_{ii} = \sum_{k} (A_{ik} \cdot B_{ki})
$$

Sea $D = A \odot B^T$ (producto element-wise $\odot$). Sabemos que $B^T_{ik} = B_{ki}$.

$$
D_{ik} = A_{ik} \cdot B^T_{ik} = A_{ik} \cdot B_{ki}
$$

Sumando sobre columnas ($axis=1$):

$$
\sum_{k} D_{ik} = \sum_{k} (A_{ik} \cdot B_{ki}) = C_{ii}
$$

<br>


**6. Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente en un nuevo modelo `EfficientQDA`.**

In [22]:
class EfficientQDA(TensorizedQDA):
    def predict(self, X):
        # Utiliza la propiedad de la diagonal para no computar la matriz (n, n)
        
        # log_dets: (k, 1)
        log_dets = 0.5 * np.log(LA.det(self.tensor_inv_cov)).reshape(-1, 1)
        
        # unbiased_x: (k, p, n)
        # Precomputar self.tensor_means si no es broadcastable directo? No, es (k,p,1), X is (p,n). ok.
        unbiased_x = X - self.tensor_means
        
        # U = Singma^-1 * (x - mu)
        # (k, p, p) @ (k, p, n) -> (k, p, n)
        u = self.tensor_inv_cov @ unbiased_x
        
        # Cuadrática: sum_features ( (x-mu) * U )
        # Element-wise mult: (k, p, n) * (k, p, n)
        # Sum axis=1 (features)
        quad_term = np.sum(unbiased_x * u, axis=1) # (k, n)
        
        log_conditionals = log_dets - 0.5 * quad_term # (k, n)
        log_posteriori = self.log_a_priori.reshape(-1, 1) + log_conditionals
        
        return np.argmax(log_posteriori, axis=0).reshape(1, -1)

**7. Comparación de performance (QDA vs Variantes)**

Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

In [ ]:
# Ejecutamos el benchmark para los modelos optimizados que acabamos de definir
b.bench(FasterQDA)
b.bench(EfficientQDA)

# Mostramos la tabla resumen comparando contra el QDA base
# Esto nos permitirá ver el speedup y el uso de memoria
b.summary(baseline='QDA')

FasterQDA (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

FasterQDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

EfficientQDA (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

EfficientQDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb
model,,,,,,,,,
FasterQDA,3.816928,1.017707,448.185581,18.432903,0.886117,0.270103,0.001993,3201.747864,0.00052
EfficientQDA,2.419501,0.329200,5.786387,0.536643,0.885303,0.268471,0.002143,38.996544,0.00000


### Comparación de performance (QDA vs Variantes)

Al comparar las variantes implementadas sin Cholesky, se observan diferencias claras entre FasterQDA y EfficientQDA respecto al QDA base:

- **FasterQDA** presenta un tiempo de testeo muy alto (~448 ms) y un consumo de memoria extremadamente elevado (~3200 MB). Esto ocurre porque construye una matriz \( n \times n \), lo que escala en \( O(n^2) \). Aunque mantiene la misma accuracy (~0.886), su costo computacional lo vuelve poco eficiente.

- **EfficientQDA**, en cambio, evita generar dicha matriz y calcula solo la diagonal, reduciendo el costo a \( O(n) \). Esto se refleja en un tiempo de testeo muy bajo (~5.8 ms) y un uso de memoria mínimo (~39 MB), manteniendo una accuracy equivalente (~0.885).

### Conclusión
Los resultados coinciden con lo esperado: FasterQDA es computacionalmente ineficiente a gran escala, mientras que EfficientQDA logra mejoras significativas en tiempo y memoria sin sacrificar precisión.



---


## Respuestas Consigna Cholesky

### Diferencias entre implementaciones de `QDA_Chol`

<br>

**8. Inversa con Cholesky**

Si $A = L L^T$, entonces $A^{-1} = (L L^T)^{-1} = (L^T)^{-1} L^{-1} = (L^{-1})^T L^{-1}$.

En la forma cuadrática $(x-\mu)^T \Sigma^{-1} (x-\mu)$, podemos reescribir $\Sigma^{-1} = (L^{-1})^T L^{-1}$.

Entonces, $(x-\mu)^T (L^{-1})^T L^{-1} (x-\mu) = (L^{-1}(x-\mu))^T (L^{-1}(x-\mu)) = ||L^{-1}(x-\mu)||^2$.

Esto es útil porque en lugar de una forma cuadrática general, calculamos la norma al cuadrado de un vector transformado $y = L^{-1}(x-\mu)$.

<br>

**9. QDA_Chol1 vs QDA**

`QDA_Chol1` descompone la matriz de covarianza $\Sigma$ usando Cholesky ($\Sigma = L L^T$), invierte $L$ explícitamente y almacena $L^{-1}$.

Para predecir, calcula $y = L^{-1}(x - \mu)$ mediante un producto matriz-vector.

El log-determinante se calcula fácilmente como $2 \sum \log(diag(L))$.

La diferencia principal con `QDA` es que usa la factorización para invertir y para calcular el determinante, lo cual es generalmente más numéricamente estable que `LA.inv` y `LA.det` sobre la matriz completa.

<br>

**10. Diferencias entre Chol1, Chol2 y Chol3**

*   **Chol1**: Calcula $L$, invierte $L$ usando `LA.inv` (genérico), guarda $L^{-1}$. Predicción: producto matricial @.
*   **Chol2**: Calcula $L$, guarda $L$. Predicción: usa `solve_triangular` para resolver $L y = (x-\mu)$. No invierte explícitamente. 

Es más estable pero más lento en predicción (resolución de sistemas vs producto matriz-vector).
*   **Chol3**: Calcula $L$, calcula $L^{-1}$ usando `dtrtri` (rutina LAPACK específica para inversas triangulares), guarda $L^{-1}$. 

Es teóricamente más eficiente/correcto que Chol1 al usar la rutina especializada para triangular.

<br>

**11. Comparación de la performance de las variantes de QDA implementadas** 

Comparar la performance de las 7 variantes de QDA implementadas hasta ahora. 

¿Qué se observa? 

¿Hay alguna de las implementaciones de `QDA_Chol` que sea claramente mejor que las demás?  

¿Alguna que sea peor?

In [24]:
b.bench(QDA_Chol1)
b.bench(QDA_Chol2)
b.bench(QDA_Chol3)
b.summary(baseline='QDA')

QDA_Chol1 (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

QDA_Chol1 (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

QDA_Chol2 (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

QDA_Chol2 (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

QDA_Chol3 (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

QDA_Chol3 (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb
model,,,,,,,,,
FasterQDA,3.816928,1.017707,448.185581,18.432903,0.886117,0.270103,0.001993,3201.747864,0.000520
EfficientQDA,2.419501,0.329200,5.786387,0.536643,0.885303,0.268471,0.002143,38.996544,0.000000
QDA_Chol1,2.752980,0.313872,326.094481,3.360117,0.884827,0.269180,0.001926,0.094749,0.000184
QDA_Chol2,2.618680,0.380376,746.496485,5.299495,0.884890,0.269524,0.002164,0.095041,0.000173
QDA_Chol3,2.671593,0.233013,323.508558,3.254473,0.884770,0.268486,0.002141,0.094604,0.000134


### Análisis de Variantes QDA implementadas

Al comparar las 7 variantes de QDA evaluadas hasta el momento (FasterQDA, EfficientQDA y las tres versiones basadas en Cholesky), se observan diferencias marcadas en tiempos de testeo, estabilidad y uso de memoria:

- **FasterQDA** vuelve a destacarse como la peor alternativa en términos de performance: su tiempo de testeo es extremadamente alto (~448 ms) y el uso de memoria durante el test sigue siendo desproporcionado (~3200 MB). Aunque mantiene buena accuracy, su costo computacional lo hace impráctico.

- **EfficientQDA** continúa siendo la variante más rápida y eficiente dentro del grupo no-Cholesky, con tan solo ~5.8 ms de testeo y consumo mínimo de memoria (~39 MB), manteniendo además una accuracy muy similar al QDA original.

### Comparación entre las variantes con Cholesky

Entre **QDA_Chol1**, **QDA_Chol2** y **QDA_Chol3** se observan diferencias claras:

- **QDA_Chol1** y **QDA_Chol3** muestran tiempos de testeo muy parecidos (~326 ms y ~323 ms respectivamente), lo que los convierte en las alternativas más rápidas dentro del grupo Cholesky. Su accuracy también es prácticamente idéntica (~0.8848).

- **QDA_Chol2**, en cambio, tiene un desempeño considerablemente peor: su tiempo de testeo se duplica respecto a Chol1/Chol3 (~746 ms), siendo la variante más lenta entre las implementaciones basadas en factorización de Cholesky. Esto coincide con lo esperado, ya que Chol2 utiliza `solve` en lugar de invertir matrices o emplear rutinas más optimizadas.

### Conclusión

- Dentro de las variantes con Cholesky, **QDA_Chol3** se posiciona como la mejor en términos de velocidad sin perder estabilidad numérica.  
- **QDA_Chol2** es claramente la peor opción del grupo por su mayor tiempo de testeo.  
- Comparando todas las variantes, **EfficientQDA** sigue siendo la más escalable y eficiente en general.


### Optimización

**12. Implementación de TensorizedChol** 

In [25]:
class TensorizedChol(QDA_Chol3):
    def _fit_params(self, X, y):
        super()._fit_params(X, y)
        self.tensor_linv = np.stack(self.L_invs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self, x):
        # x is (p, 1)
        # unbiased_x: (k, p, 1)
        unbiased_x = x - self.tensor_means
        # y = L^-1 (x-mu) -> (k, p, p) @ (k, p, 1) -> (k, p, 1)
        y_trans = self.tensor_linv @ unbiased_x 
        dist = np.sum(y_trans**2, axis=1).flatten() # (k,)
        
        diags = np.diagonal(self.tensor_linv, axis1=1, axis2=2)
        log_dets = 2 * np.sum(np.log(diags), axis=1)
        
        return log_dets - 0.5 * dist

    def _predict_one(self, x):
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

**13. Implementación de EfficientChol** 

In [26]:
class EfficientChol(TensorizedChol):
    def predict(self, X):
        # Vectorized over classes AND observations
        
        diags = np.diagonal(self.tensor_linv, axis1=1, axis2=2)
        log_dets = 2 * np.sum(np.log(diags), axis=1).reshape(-1, 1) # (k, 1)
        
        # Broadcasting automatically handles the shapes
        # self.tensor_means: (k, p, 1)
        # X: (p, n) -> broadcasts to (1, p, n)
        unbiased_x = X - self.tensor_means # Result: (k, p, n)
        
        # y = L^-1 (x - mu)
        # self.tensor_linv: (k, p, p)
        # unbiased_x: (k, p, n)
        # Matrix multiplication: (k, p, p) @ (k, p, n) -> (k, p, n)
        y_trans = self.tensor_linv @ unbiased_x 
        
        # Distance squared: ||y||^2 
        dist = np.sum(y_trans**2, axis=1) # (k, n)
        
        log_conditionals = log_dets - 0.5 * dist
        log_posteriori = self.log_a_priori.reshape(-1, 1) + log_conditionals
        
        return np.argmax(log_posteriori, axis=0).reshape(1, -1)

**14. Comparación de performance de las 9 variantes de QDA implementadas**

Comparar la performance de las 9 variantes de QDA implementadas. 

¿Qué se observa? 

A modo de opinión ¿Se condice con lo esperado?

In [27]:
b.bench(TensorizedChol)
b.bench(EfficientChol)
b.summary(baseline='QDA')

TensorizedChol (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

TensorizedChol (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

EfficientChol (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

EfficientChol (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb
model,,,,,,,,,
FasterQDA,3.816928,1.017707,448.185581,18.432903,0.886117,0.270103,0.001993,3201.747864,0.000520
EfficientQDA,2.419501,0.329200,5.786387,0.536643,0.885303,0.268471,0.002143,38.996544,0.000000
QDA_Chol1,2.752980,0.313872,326.094481,3.360117,0.884827,0.269180,0.001926,0.094749,0.000184
QDA_Chol2,2.618680,0.380376,746.496485,5.299495,0.884890,0.269524,0.002164,0.095041,0.000173
QDA_Chol3,2.671593,0.233013,323.508558,3.254473,0.884770,0.268486,0.002141,0.094604,0.000134
TensorizedChol,2.444089,0.378680,35.097663,1.616923,0.866425,0.268913,0.001966,0.160751,0.000192
EfficientChol,2.349084,0.091210,4.928213,0.595650,0.867233,0.268616,0.001848,38.996742,0.000134


## **Análisis Final de las 9 Variantes**

Al comparar todas las variantes desarrolladas (QDA base, TensorizedQDA, FasterQDA, EfficientQDA, las tres QDA_Chol y las dos versiones tensorizadas con Cholesky), se observan diferencias claras tanto en tiempo de testeo como en uso de memoria y accuracy.

### **Comportamiento general**

- **FasterQDA** vuelve a ser la peor alternativa: su tiempo de testeo (~448 ms) y su uso de memoria (~3200 MB) lo hacen impracticable para datasets moderados o grandes, aunque su accuracy se mantiene correcta.

- **EfficientQDA** y **EfficientChol** son las variantes más rápidas del conjunto.  
  - **EfficientQDA**: ~5.7 ms  
  - **EfficientChol**: ~4.9 ms (la más rápida entre todas)  
  Ambas mantienen una memoria muy baja (~39 MB para EfficientChol) y una accuracy cercana a la del modelo original.

- Las versiones **QDA_Chol1** y **QDA_Chol3** presentan rendimientos similares, con tiempos de test entre ~323 y ~326 ms y accuracies alrededor de 0.8847. Son significativamente más lentas que las versiones “Efficient”, pero mucho más estables y livianas que FasterQDA.

- **QDA_Chol2** es la peor implementación dentro del grupo Cholesky, con un tiempo de testeo muy superior (~746 ms). Esto coincide con el uso de `solve` en lugar de invertir o usar rutinas más optimizadas.

- Entre las variantes tensorizadas con Cholesky:
  - **TensorizedChol** mejora el tiempo respecto a Chol1/2/3 (~35 ms), pero sigue siendo más lento que EfficientChol.  
  - Su accuracy cae levemente (~0.866), lo que muestra un pequeño costo numérico asociado a esta estrategia.

<br>

## **Conclusión**

Los resultados coinciden con lo esperado teóricamente:

- Evitar construir matrices completas y aprovechar propiedades algebraicas lleva a mejoras drásticas en performance (**EfficientQDA** y **EfficientChol** son los claros ganadores).
- Las variantes basadas en **Cholesky** mejoran estabilidad numérica, pero dependiendo de la implementación pueden ser más lentas.
- **EfficientChol** logra el mejor compromiso global entre rapidez, bajo uso de memoria y estabilidad.

En resumen, las optimizaciones `EfficientQDA / EfficientChol` superan ampliamente a las demás variantes, mientras que **FasterQDA** confirma que eliminar bucles sin considerar la complejidad real puede empeorar mucho el rendimiento.